In [1]:
import pandas as pd

In [2]:
from tensorflow.keras import Sequential

In [26]:
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D, Embedding, SimpleRNN

In [39]:
from tensorflow.keras.optimizers import Adam

In [121]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [114]:
from tensorflow.keras.utils import to_categorical, pad_sequences

In [37]:
from sklearn.model_selection import train_test_split

In [5]:
from nltk import FreqDist

In [6]:
from nltk.tokenize import word_tokenize

In [7]:
import tensorflow as tf

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [68]:
import numpy as np

In [9]:
df = pd.read_csv("/git/dados/teste/ecommerceDataset.csv", header=None, names=["class", "description"])

In [10]:
# dicionario = { "bom": 1000, "dia":  987, "ola": 435 ... }

In [11]:
def gerar_dicionario( serie, dicionario_inicial = {}, inicial_contador = 0 ):
    maior_frase = 0
    numero_palavra = inicial_contador
    for celula in serie:
        if type(celula) == str:
            celula_tokens = celula.split(" ")
            if len(celula_tokens) > maior_frase:
                maior_frase = len(celula_tokens)
            for palavra in celula_tokens:
                if palavra not in dicionario_inicial:
                    dicionario_inicial[palavra] = numero_palavra
                    numero_palavra += 1
    return maior_frase, dicionario_inicial

In [12]:
maior, dicionario = gerar_dicionario(df["description"], { "<PAD>" : 0 }, 1)

In [14]:
def transforma_texto_numeros( texto ):
    global dicionario
    lista_palavras_numericas = []
    if type(texto) is str:
        tokens = texto.split(" ")
        for token in tokens:
            if token in dicionario:
                numero_palavra = dicionario[token]
                lista_palavras_numericas.append(numero_palavra)
    return lista_palavras_numericas
    

In [125]:
def limpar_textos( texto ):
    if type(texto) == str:
        return texto
    else:
        return ""

In [164]:
def truncar_textos( texto ):
    if type(texto) == str:
        tokens = texto.split(" ")
        return " ".join(tokens[0:500])
    else:
        return ""

In [165]:
df["description_clean"] = df["description"].apply(truncar_textos)

In [174]:
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")

In [175]:
tokenizer.fit_on_texts( df["description_clean"].to_list() )

In [176]:
len(tokenizer.word_index)

87866

In [177]:
X = tokenizer.texts_to_sequences(df["description_clean"].to_list())
X

[[344,
  1,
  53,
  1,
  143,
  708,
  1,
  236,
  413,
  400,
  1,
  96,
  100,
  34,
  96,
  100,
  99,
  35,
  4,
  49,
  1,
  48,
  40,
  8,
  1,
  511,
  9,
  948,
  1,
  377,
  44,
  255,
  229,
  1,
  3,
  1,
  1,
  13,
  14,
  10,
  25,
  358,
  411,
  4,
  1,
  44,
  104,
  11,
  143,
  103,
  233,
  3,
  255,
  6,
  1,
  320,
  14,
  1,
  10,
  547,
  5,
  1,
  12,
  462,
  23,
  1,
  5,
  1,
  14,
  239,
  1,
  17,
  10,
  6,
  1,
  1,
  69,
  36,
  114,
  2,
  78,
  323,
  3,
  762,
  1,
  185,
  15,
  55,
  1,
  9,
  114,
  2,
  3,
  1,
  3,
  1,
  1,
  1,
  3,
  719,
  1,
  14,
  829,
  310,
  1,
  1,
  1,
  8,
  190,
  3,
  1,
  518,
  1,
  4,
  2,
  1,
  17,
  536,
  6,
  1,
  69,
  377,
  1,
  9,
  115,
  1,
  101,
  1,
  5,
  1,
  1,
  3,
  889,
  865,
  497,
  5,
  124,
  41,
  352,
  4,
  948,
  1,
  55,
  1,
  1,
  124,
  233,
  865,
  7,
  226,
  132,
  398,
  109,
  3,
  91,
  5,
  11,
  360,
  212,
  9,
  14,
  1,
  548,
  1,
  428,
  22,
  7,
  1,
  3,
  428,
 

In [184]:
X_padded = pad_sequences(X, dtype='int32', padding='post', value=0)
X_padded.shape

(50425, 577)

In [73]:
# df["description"][10]

In [72]:
# transforma_texto_numeros( df["description"][10] ) 

In [185]:
# df["description_numbers"] = df["description"].apply(transforma_texto_numeros)

In [19]:
maior_lista = 0
for lista_palavras in df["description_numbers"]:
    if len(lista_palavras) > maior_lista:
        maior_lista = len(lista_palavras)
maior_lista

8183

In [103]:
def padding_direita( lista_palavras ):
    global maior_lista
    quantidade = maior_lista - len(lista_palavras)
    for _ in range(quantidade):
        lista_palavras.append( 0 )
    # palavras_padded = np.asarray(lista_palavras).astype('int32')
    #tf.convert_to_tensor(palavras_padded, dtype=tf.int32)
    return lista_palavras

In [186]:
# X = pad_sequences(df["description_numbers"], dtype='int32', padding='post', value=0)
# X

In [116]:
# df["description_numbers_padded"] = df["description_numbers"].apply(padding_direita)

In [117]:
# X = df["description_numbers_padded"].to_list()
# X

In [187]:
df["class"].unique()

array(['Household', 'Books', 'Clothing & Accessories', 'Electronics'],
      dtype=object)

In [188]:
df["class_categorical"] = df["class"].replace({'Household': 0, 'Books': 1, 'Clothing & Accessories': 2, 'Electronics': 3})

C:\Users\anton\AppData\Local\Temp\ipykernel_9456\495202090.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["class_categorical"] = df["class"].replace({'Household': 0, 'Books': 1, 'Clothing & Accessories': 2, 'Electronics': 3})


In [189]:
Y = to_categorical(df["class_categorical"], 4)

In [60]:
Y

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [190]:
modelo = Sequential()
modelo.add( Embedding(len(tokenizer.word_index), 64) )
modelo.add( SimpleRNN( 12 ) )
modelo.add( Dense( 4 , activation="softmax") )
modelo.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_8 (SimpleRNN)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [191]:
adam = Adam(learning_rate=0.01)
modelo.compile( optimizer=adam, loss="categorical_crossentropy", metrics=["Accuracy"])

In [192]:
modelo.fit(X_padded, Y, epochs=5, batch_size=64)

Epoch 1/5
788/788 ━━━━━━━━━━━━━━━━━━━━ 186s 229ms/step - Accuracy: 0.3792 - loss: 1.3447
Epoch 2/5
788/788 ━━━━━━━━━━━━━━━━━━━━ 204s 258ms/step - Accuracy: 0.3945 - loss: 1.3398
Epoch 3/5
788/788 ━━━━━━━━━━━━━━━━━━━━ 221s 280ms/step - Accuracy: 0.4009 - loss: 1.3353
Epoch 4/5
788/788 ━━━━━━━━━━━━━━━━━━━━ 194s 246ms/step - Accuracy: 0.4015 - loss: 1.3322
Epoch 5/5
788/788 ━━━━━━━━━━━━━━━━━━━━ 187s 237ms/step - Accuracy: 0.3953 - loss: 1.3366


In [229]:
row = 35000
print("Class: ", df["class"][row], ' - ', df["class_categorical"][row], ' - ', Y[row])
print("Text: ", df["description_clean"][row])
X_prev = tokenizer.texts_to_sequences( [ df["description_clean"][row] ] )
X_prev_padded = pad_sequences(X_prev, dtype='int32', padding='post', value=0, maxlen=577)
Y_pred = modelo.predict( X_prev_padded )
to_categorical(np.argmax(Y_pred, axis = 1))

Class:  Clothing & Accessories  -  2  -  [0. 0. 1. 0.]
Text:  PUMA Men's Athletic Socks Designed to project a sporty and casual appeal, puma’s latest collection of socks promises to make work-outs more fashionable. Revel in the brand’s unique range of trendsetting lollipop neons, cuffed capris and roomy backpacks.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step


array([[1.]])

(1, 577)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


array([[1.]])